In [ ]:
import os
from bin.myinfer import InferModel

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
model = InferModel(checkpoint="exp/valle/best-valid-loss.pt")

In [1]:
import sys

if "bin.utils" in sys.modules:
    del sys.modules["bin.utils"]

from bin.utils import *


class AIShell2DataWrapper:
    def __init__(self) -> None:
        dir = Path("download/aishell2/AISHELL-2/iOS/test")
        lines = open(dir / "trans.txt").readlines()
        self.voice = {
            key: value for line in lines for key, value in [self.line_parser(line)]
        }

    def line_parser(self, line: str):
        id = line[:11]
        id1 = line[1:6]
        # id2 = line[6:11]
        text = line[12:].strip()
        return id, {
            "file_name": Path(f"wav/{id1}/{id}.wav"),
            "text": text,
            "npc_id": int(id1[-2:]),
        }

    def get_ids(
        self,
        count: int = 5,
        min_len: int = 8,
        max_len: int = 18,
        nice: bool = False,
    ):
        ids = [
            key
            for key in self.voice.keys()
            if min_len <= len(self.voice[key]["text"]) <= max_len
        ]
        if nice:
            nice_set = {16, 17, 19, 22, 23, 25}
            ids = [key for key in ids if int(self.voice[key]["id"]) in nice_set]
        return random_samples(ids, count)

    def create_file(
        self,
        file_name: PathLike,
        ids,
        texts,
        infer_dir="audios/aishell2",
        copy: bool = False,
    ):
        voice = self.voice
        return create_file_base(
            file_name=file_name,
            ids=ids,
            get_info=lambda id: (voice[id]["text"], voice[id]["file_name"]),
            texts=texts,
            infer_dir=infer_dir,
            copy=copy,
        )

In [ ]:
wrapper = AIShell2DataWrapper()

ids = wrapper.get_ids(count=5,nice=True)

texts = get_tts_texts('short')

def process_char(c):
    if re.match(r'[\u4e00-\u9fff，]',c):
        return c
    else:
        return ''
def process_str(s):
    return "".join([process_char(x) for x in s])

texts = [process_str(x) for x in texts]

file_name = wrapper.create_file("aishell2.txt", ids, texts)

# others

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt

filename = "audios/aishell2/IT0016W0441.wav"
waveform, sr = librosa.load(filename)

plt.figure(figsize=(12, 4))
# librosa.display.waveplot(waveform, sr=sr)
plt.plot(waveform, color="#007acc")
plt.title("Waveform")
plt.xlabel("Time (seconds)")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
from collections import Counter


wrapper = AIShell2DataWrapper()
counter = Counter()
count = 0
with open("download/aishell2/AISHELL-2/iOS/data/trans.txt") as file:
    while True:
        line = file.readline()
        if not line:
            break
        ls = re.findall("[㶧䶮𫚉]+", line[12:])
        count += bool(ls)
        counter.update(ls)

sorted_counter = sorted(counter.items(), key=lambda x: x[1], reverse=True)

print(f"count:{count}")
print(*sorted_counter, sep="\n")

In [ ]:
# from valle.data.collation import get_text_token_collater
# from valle.data.tokenizer import TextTokenizer, tokenize_text


# text_tokenizer = TextTokenizer(backend="pypinyin_initials_finals")
# text_collater = get_text_token_collater("data/tokenized/unique_text_tokens.k2symbols")
# text = "WE WILL ROCK YOU 好而"


# print([tokenize_text(text_tokenizer, text=text)])

In [ ]:
from bin.utils import *
import re

for item in read_jsonl_gz('data/manifests/aishell2_supervisions_train.jsonl.gz'):
    ls = re.findall(r'[a-zA-Z]+',item['text'])
    if ls:
        print(ls)